In [ ]:


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType
from pyspark.sql.functions import col, window

spark = SparkSession.builder.appName("StreamingApp").getOrCreate()


schema = StructType() \
    .add("id", IntegerType(),True),
    .add("name", StringType(),True),
    .add("age", IntegerType(),True),
    .add("location", StringType(),True),
    .add("event_time", TimestampType(),True)

df = spark.readStream.schema(schema).option("header", "true").csv("./class")
#Windowing
df_watermark = df.withWatermark("event_time", "1 minute")
event_count = df_watermark.groupBy(window(col("event_time"),"1 minute"), col("location")).count() #10:00 - 10:01, 10:01 - 10:02 / 10:00 - 10:10, 10:05 - 10:15, 10:10 - 10:20
event_count_flattened = event_count.select(
    col("window.start").alias("window_start"),
    col("window.end").alias("window_end"),
    col("location"),
    col("count")
)
query = event_count_flattened.writeStream.format("csv").option("path","./output").option("checkpointLocation","./windowcheckpoint").start()


In [1]:
import os
print("HADOOP_HOME =", os.environ.get("HADOOP_HOME"))


HADOOP_HOME = C:\Hadoop
